# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [2]:
from google.colab import userdata
from openai import OpenAI
import os

# API_KEY 를 명시적으로 전달하는 방법 - 이걸 더 추천!
OPENAI_API_KEY = userdata.get('MY_OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# 환경변수에서 API_KEY 를 전달하는 방법
# os.environ("OPENAI_API_KEY") = userdata.get("OPENAI_API_KEY")
# client = OpenAI()

In [ ]:
model           = "gpt-4o-mini"
system_message  = "너는 친절한 챗봇이야."
prompt          = "안녕, 나는 LLM 꿈나무 정해인이야."
answer          = "안녕하세요. 정해인 님. 만나서 반가워요. LLM 꿈나무라니 멋지네요."
params = {
  "model"                 : model,
  "messages"              : [
                              { "role": "system"    , "content": [{"type": "text", "text": system_message}] },
                              { "role": "user"      , "content": [{"type": "text", "text": prompt}] },
                              { "role": "assistant" , "content": [{"type": "text", "text": answer}] },
                              { "role": "user"      , "content": [{"type": "text", "text": "Transformer 를 공부하고싶어."}] },
                            ],
  "response_format"       : {"type": "text"},
  "temperature"           : temperature,
  "max_completion_tokens" : 2048,
  "top_p"                 : top_p,
  "frequency_penalty"     : 0,
  "presence_penalty"      : 0
}
response = client.chat.completions.create()

## 챗봇이 이전 대화내용을 기억하게 만들려면,
이전에 주고받은 메시지를 누적하면서 반복처리를 해야 함.

In [3]:
# 대화내역을 로깅
messages = [
    {"role":"system", "content":"너는 친절한 챗봇이야."}
]

def add_msg(role, msg):
  messages.append({ "role":role, "content":f"""{msg}""" })

print("종료하려면, exit 를 입력하세요.")
while True:
  # 사용자 입력
  user_input = input("User : ")
  if user_input.strip().lower() == "exit":
    print("채팅을 종료합니다.")
    break

  # 사용자 입력을 messages 에 추가.
  add_msg("user", user_input)

  # LLM 요청
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      temperature=1
  )

  assistant_msg = response.choices[0].message.content
  print("Assistant : ", assistant_msg)

  add_msg("assistant", assistant_msg)

종료하려면, exit 를 입력하세요.
User : 난 우주존잘 정해인이야
Assistant :  안녕하세요, 정해인님! 우주존잘이라니 멋지네요! 당신은 우주와 관련된 어떤 것들을 좋아하시나요? 혹시 우주에 관한 이야기를 나누고 싶으신가요?
User : Transformer 를 중학생도 이해할 수 있게 쉽게 설명하고 예시도 들어줘
Assistant :  물론이죠! Transformer는 주로 자연어 처리(NLP)에서 사용하는 강력한 인공지능 모델입니다. 복잡하게 들릴 수 있지만, 간단하게 설명해볼게요.

### Transformer란?
Transformer는 텍스트와 같은 데이터의 의미를 이해하고 처리하는 데 도움을 주는 일종의 컴퓨터 프로그램입니다. 이 모델은 단어들 간의 관계를 잘 이해하고, 문장의 맥락을 파악하는 데 특화되어 있어요.

### 어떻게 작동하나요?
Transformer는 몇 가지 중요한 부분으로 이루어져 있어요:

1. **입력 데이터**: 텍스트(문장)를 입력으로 받습니다.
2. **Self-Attention**: 문장 안의 각 단어가 다른 단어들과 어떻게 관련되어 있는지를 판단합니다. 예를 들어, "나는 사과를 좋아해. 사과는 맛있어."에서 "사과"라는 단어가 두 번 나올 때, 이 두 단어가 서로 관련성을 갖도록 만들어줍니다.
3. **인코더와 디코더**: Transformer는 보통 인코더(입력 정보를 이해하는 역할)와 디코더(출력 정보를 생성하는 역할)로 나뉩니다. 예를 들어, 하나의 문장을 다른 언어로 번역하는 데 사용될 수 있습니다.

### 예시
Imagine you want to translate the sentence "I love apples" into Korean. Here's how a Transformer would work:

1. **인코딩**: "I love apples"라는 문장을 입력으로 받아서 각 단어의 의미를 이해합니다.
2. **Attention Mechanism**: 여기서 "사랑하다"와 "사과"라는 단

## Stream 형식으로 응답을 출력해서 대기시간 줄여보기

In [4]:
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"user", "content":"stream 테스트를 할거야. 적당히 긴 텍스트를 보내줘."},
    ],
    stream=True
)
for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(content, end="")

물론입니다! 아래는 적당히 긴 텍스트입니다.

---

한때, 작은 마을에 살던 한 소년이 있었습니다. 그는 매일 아침 해가 뜨기 전에 일어나, 그의 가장 좋아하는 장소인 숲으로 향했습니다. 숲은 그에게 신비로운 세계처럼 느껴졌고, 나무들은 높고 울창하게 자라있었으며, 다양한 생물들이 저마다의 방식으로 그곳에서 생존하고 있었습니다.

소년은 숲 속에서 새로운 친구들을 많이 만났습니다. 개구리, 다람쥐, 그리고 때때로 지나가는 사슴까지도 그의 친구가 되었습니다. 그는 자연과 하나가 되는 듯한 느낌을 받으며, 다양한 동식물들을 관찰하고, 그들의 생활 방식을 배우는 것을 즐겼습니다.

어느 날, 소년은 숲 속 깊은 곳에서 반짝이는 작은 연못을 발견했습니다. 그 연못은 맑고 투명한 물이 가득 차 있었고, 햇빛이 비치면 물속의 작은 물고기들이 반짝이며 수면 위로 올라오는 모습이 매우 아름다웠습니다. 그는 그곳에서 시간을 보내며 생각에 잠기곤 했습니다. “이 연못은 왜 이렇게 아름답지? 자연은 정말로 신비로운 것 같아.”

시간이 흐르면서 소년은 더 많은 것을 배우고 경험하게 되었습니다. 그는 자연의 소중함을 깨닫고, 지구를 보호해야 한다는 책임감도 느끼게 되었습니다. 그래서 그는 자신의 마을로 돌아와 친구들에게 숲의 아름다움과 그곳에서 배운 것들을 이야기하기 시작했습니다. 

그의 이야기를 들은 친구들은 숲에 대한 흥미를 느끼게 되었고, 함께 숲 탐험을 나가기 시작했습니다. 그들은 나무를 심고, 쓰레기를 줍는 등의 활동을 통해 자연을 보호하는 방법을 익혔습니다. 소년은 이제 자신이 사랑하는 숲을 지키는 작은 수호자가 된 것이었습니다.

마을 사람들은 소년과 그의 친구들을 보며 감명을 받았고, 점차 마을 전체가 자연 보호에 대한 관심을 가지게 되었습니다. 이렇게 한 소년의 작은 행동이 점차 큰 변화를 만들어내며, 마을 사람들은 더 나은 세상을 만들기 위해 함께 노력하게 되었습니다.

---

이 이야기는 자연의 아름다움과 그 소중함을 깨닫게 하는 내용입니다. 도움이 되었

## 채팅봇 stream 버전

In [6]:
# 대화내역을 로깅
messages = [
    {"role":"system", "content":"너는 친절한 챗봇이야."}
]

def add_msg(role, msg):
  messages.append({ "role":role, "content":f"""{msg}""" })

print("종료하려면, exit 를 입력하세요.")
while True:
  # 사용자 입력
  user_input = input("User : ")
  if user_input.strip().lower() == "exit":
    print("채팅을 종료합니다.")
    break

  # 사용자 입력을 messages 에 추가.
  add_msg("user", user_input)

  # LLM 요청
  stream = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      stream=True
  )
  chunks = []
  for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
      print(content, end="", flush=True)  # flush=True :: 내부 buffer 를 사용하지 않는다는 뜻.
      chunks.append(content)

  assistant_msg = "".join(chunks)
  print("Assistant : ", assistant_msg)

  add_msg("assistant", assistant_msg)

종료하려면, exit 를 입력하세요.
User : 내 이름은 지훈이야. 지금은 LLM api 활용을 배우고있어
안녕하세요, 지훈님! LLM API 활용을 배우고 계시다니 멋지네요. 궁금한 점이나 도움이 필요한 부분이 있으면 언제든지 말씀해 주세요!Assistant :  안녕하세요, 지훈님! LLM API 활용을 배우고 계시다니 멋지네요. 궁금한 점이나 도움이 필요한 부분이 있으면 언제든지 말씀해 주세요!
User : OpenAI 의 API 를 stream 으로 받아 출력하는거 굉장히 쉽게 잘 만들었네. 칭찬해.
칭찬해 주셔서 감사합니다, 지훈님! OpenAI의 API는 정말 유용하고 강력한 도구입니다. 스트리밍 기능은 실시간으로 응답을 받을 수 있어 더욱 편리하죠. 혹시 API 사용 중에 궁금한 점이나 도움이 필요하신 부분이 있으면 언제든지 물어보세요!Assistant :  칭찬해 주셔서 감사합니다, 지훈님! OpenAI의 API는 정말 유용하고 강력한 도구입니다. 스트리밍 기능은 실시간으로 응답을 받을 수 있어 더욱 편리하죠. 혹시 API 사용 중에 궁금한 점이나 도움이 필요하신 부분이 있으면 언제든지 물어보세요!
User : exit
채팅을 종료합니다.


# Token counting
비용 = (입력 토큰수 * 입력단가) + (출력 토큰수 * 출력단가) * 월 서비스 호출수

In [7]:
# 토큰수를 세어주는 패키지
!pip install tiktoken

In [9]:
# 각 모델에 따른 토크나이져(인코딩) 가져오기. OpenAI 에서 만든거
import tiktoken

gpt35      = tiktoken.encoding_for_model("gpt-3.5")
gpt4o_mini = tiktoken.encoding_for_model("gpt-4o-mini")
# gpt41      = tiktoken.encoding_for_model("gpt-4.1")   # 아직 코드 업데이트가 안되서 오류남.
gpt41      = tiktoken.get_encoding("cl100k_base")

print("gpt35      : ", gpt35)
print("gpt4o_mini : ", gpt4o_mini)

gpt35      :  <Encoding 'cl100k_base'>
gpt4o_mini :  <Encoding 'o200k_base'>


In [11]:
# 토큰수 세기
text = "아버지가 방에 들어가신다"
print("gpt35 encode      : ", gpt35.encode(text))
print("gpt4o_mini encode : ", gpt4o_mini.encode(text))

print("gpt35 len      : ", len(gpt35.encode(text)))
print("gpt4o_mini len : ", len(gpt4o_mini.encode(text)))

gpt35 encode      :  [54059, 80104, 22035, 20565, 75908, 19954, 56938, 97, 32179, 20565, 83628, 13447]
gpt4o_mini encode :  [7653, 24963, 158317, 24030, 3107, 57217, 4081, 11753, 2276]
gpt35 len      :  12
gpt4o_mini len :  9


## 토큰 비용 계산하기

In [12]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_35      = gpt35.encode(text)
encoded_text_4o_mini = gpt4o_mini.encode(text)

print("gpt-3.5 토큰 수     : ", len(encoded_text_35))
print("gpt-4o-mini 토큰 수 : ", len(encoded_text_4o_mini))

gpt-3.5 토큰 수     :  1215
gpt-4o-mini 토큰 수 :  703


In [17]:
response_gpt4o = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role":"system", "content":"너는 똑부러지는 시사/경제 전문가로서, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야."},
        {"role":"user", "content":text}
    ],
    temperature=0.2
)
response_4o_mini = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"너는 똑부러지는 시사/경제 전문가로서, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야."},
        {"role":"user", "content":text}
    ],
    temperature=0.2
)

output_gpt4o       = response_gpt4o.choices[0].message.content
output_gpt_4o_mini = response_4o_mini.choices[0].message.content

print("gpt-4o 응답      : ", output_gpt4o)
print("gpt-4o-mini 응답 : ", output_gpt_4o_mini, "\n")

print("gpt-4o 토큰수      : ", len(gpt35.encode(output_gpt4o)))
print("gpt-4o-mini 토큰수 : ", len(gpt4o_mini.encode(output_gpt_4o_mini)))

gpt-4o 응답      :  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 윤 대통령의 탄핵소추안 표결 시점을 앞당기는 방안을 검토 중입니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보였습니다. 민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 당초 예정된 7일 오후 7시에서 5시로 앞당기는 방안도 논의되고 있습니다. 민주당은 국민의힘의 지연 전략을 고려해 표결 시간을 조정할 가능성을 열어두고 있습니다.
gpt-4o-mini 응답 :  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령의 탄핵에 사실상 찬성 입장을 시사하자, 7일로 예정된 탄핵소추안 표결을 앞당기는 방안을 검토하고 있다. 이재명 민주당 대표는 한 대표의 입장을 신중히 파악해야 한다고 강조하며, 탄핵 표결을 서두를 필요성에 의문을 제기했다.

민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 탄핵소추안 표결 시점을 조정할 가능성도 논의되고 있다. 노종면 원내대변인은 긴장감이 높아지고 있으며, 윤 대통령을 옹호하는 세력의 반응에 대한 우려를 표명했다.

현재 민주당은 7일 오후 7시로 예정된 표결을 2시간 앞당겨 오후 5시에 진행하는 방안을 고려하고 있으며, 이는 국민의힘의 지연 전략을 염두에 둔 결정이다. 

gpt-4o 토큰수      :  298
gpt-4o-mini 토큰수 :  241


In [20]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
  encoding = tiktoken.encoding_for_model(model)
  encoded = encoding.encode(text)
  return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
  input_tokens = count_tokens(input_text, model)
  output_tokens = count_tokens(output_text, model)

  # 모델별 단가 가져오기
  price = PRICING[model]

  # 비용계산
  input_cost  = (input_tokens  / 1_000_000) * price["input"]
  output_cost = (output_tokens / 1_000_000) * price["output"]
  total_cost  = (input_cost + output_cost) * num_service_call
  return total_cost

print( "$", calc_cost(text, output_gpt4o, model="gpt-4o") )
print( "$", calc_cost(text, output_gpt_4o_mini, model="gpt-4o-mini") )

$ 3477.5
$ 250.04999999999998
